# Paquetes y librerias

In [ ]:
install.packages("ISLR")
install.packages("tidyverse")
install.packages("Hmisc")
install.packages("sna")
install.packages("igraph")
install.packages("network")
install.packages("readr")
install.packages("ergm")
install.packages("tergm")

In [ ]:
library(ISLR)
library(tidyverse)
library(Hmisc)
library(sna)
library(network)
library(readr)
library(ergm)
library(tergm)

In [ ]:
system('apt-get install libglpk-dev libgmp-dev libxml2-dev', intern=T)
install.packages("igraph")
library("igraph")

# Datos

In [10]:
url1 <- "https://raw.githubusercontent.com/oapatinop/Anlytics_redes_sociales/main/nodos2.csv"
nodos <- read.csv(url1, sep=",")

In [11]:
url2 <- "https://raw.githubusercontent.com/oapatinop/Anlytics_redes_sociales/main/arcos2.csv"
arcos <- read.csv(url2, sep=",")

In [13]:
str(arcos)

'data.frame':	851 obs. of  3 variables:
 $ X     : int  0 1 2 3 9 10 11 12 13 28 ...
 $ Source: chr  "IDG Capital" " Prosus & Naspers" " Lippo Group" " PCCW" ...
 $ Target: chr  "Tencent" "Tencent" "Tencent" "Tencent" ...


In [12]:
str(nodos)

'data.frame':	152 obs. of  33 variables:
 $ Organization.Name                           : chr  "Tencent" "Ubisoft" "Techstars" "Zomato" ...
 $ Headquarters.Location                       : chr  "Shenzhen, Guangdong, China" "Montreuil-sous-bois, Ile-de-France, France" "Boulder, Colorado, United States" "Gurgaon, Haryana, India" ...
 $ Operating.Status                            : chr  "Active" "Active" "Active" "Active" ...
 $ Company.Type                                : chr  "For Profit" "For Profit" "For Profit" "For Profit" ...
 $ Number.of.Investments                       : num  693 9 3.57 15 29 ...
 $ Number.of.Lead.Investments                  : num  375 NA 491 13 7 2 2 17 2 122 ...
 $ Number.of.Diversity.Investments             : num  15 NA 795 NA NA NA 4 2 NA 1 ...
 $ Number.of.Exits                             : num  125 1 363 2 14 2 2 15 NA 11 ...
 $ Number.of.Lead.Investors                    : num  1 1 1 12 1 15 7 NA 5 NA ...
 $ Number.of.Investors                         

In [28]:
summary(nodos)

 Organization.Name  Headquarters.Location Operating.Status   Company.Type      
 Length:152         Length:152            Length:152         Length:152        
 Class :character   Class :character      Class :character   Class :character  
 Mode  :character   Mode  :character      Mode  :character   Mode  :character  
                                                                               
                                                                               
                                                                               
                                                                               
 Number.of.Investments Number.of.Lead.Investments
 Min.   :  1.00        Min.   :  1.00            
 1st Qu.:  6.00        1st Qu.:  4.00            
 Median : 24.00        Median : 10.00            
 Mean   : 80.84        Mean   : 32.33            
 3rd Qu.: 82.00        3rd Qu.: 32.00            
 Max.   :693.00        Max.   :491.00            
 NA's   :1

In [ ]:
#df2 <- as.data.frame(df)

In [ ]:
#df2[] <- data.matrix(df2)

# Regresión QAP

In [25]:
arcos2 <- arcos[,-c(1)]
grafo <- graph_from_edgelist(as.matrix(arcos2))

In [26]:
y<- as.matrix(as_adjacency_matrix(grafo))

In [27]:
str(nodos)

'data.frame':	152 obs. of  33 variables:
 $ Organization.Name                           : chr  "Tencent" "Ubisoft" "Techstars" "Zomato" ...
 $ Headquarters.Location                       : chr  "Shenzhen, Guangdong, China" "Montreuil-sous-bois, Ile-de-France, France" "Boulder, Colorado, United States" "Gurgaon, Haryana, India" ...
 $ Operating.Status                            : chr  "Active" "Active" "Active" "Active" ...
 $ Company.Type                                : chr  "For Profit" "For Profit" "For Profit" "For Profit" ...
 $ Number.of.Investments                       : num  693 9 3.57 15 29 ...
 $ Number.of.Lead.Investments                  : num  375 NA 491 13 7 2 2 17 2 122 ...
 $ Number.of.Diversity.Investments             : num  15 NA 795 NA NA NA 4 2 NA 1 ...
 $ Number.of.Exits                             : num  125 1 363 2 14 2 2 15 NA 11 ...
 $ Number.of.Lead.Investors                    : num  1 1 1 12 1 15 7 NA 5 NA ...
 $ Number.of.Investors                         

Para el primer caso usamos las siguientes variables
*  Total.Funding.Amount.Currency..in.USD.
*  Number.of.Events
*  Number.of.Funding.Rounds
*  Number.of.Investors

In [53]:
nodos <- nodos %>%
        mutate(Total.Funding.Amount.Currency..in.USD. = ifelse(is.na(nodos$Total.Funding.Amount.Currency..in.USD.), mean(nodos$Total.Funding.Amount.Currency..in.USD., na.rm = TRUE), nodos$Total.Funding.Amount.Currency..in.USD.))

In [ ]:
fondosTotalesUSD <- (nodos$Total.Funding.Amount.Currency..in.USD.-min(nodos$Total.Funding.Amount.Currency..in.USD.))/(max(nodos$Total.Funding.Amount.Currency..in.USD.)-min(nodos$Total.Funding.Amount.Currency..in.USD.))
x1 <- matrix(0,nrow(nodos),nrow(nodos))
for(i in 1:nrow(nodos)){
  for(j in 1:nrow(nodos)){
    x1[i,j] <- 1- abs(fondosTotalesUSD[i]-fondosTotalesUSD[j])
  }
}

In [65]:
nodos <- nodos %>%
        mutate(Number.of.Events = ifelse(is.na(nodos$Number.of.Events), mean(nodos$Number.of.Events, na.rm = TRUE), nodos$Number.of.Events))
numberOfEvents <- (nodos$Number.of.Events-min(nodos$Number.of.Events))/(max(nodos$Number.of.Events)-min(nodos$Number.of.Events))
x2 <- matrix(0,nrow(nodos),nrow(nodos))
for(i in 1:nrow(nodos)){
  for(j in 1:nrow(nodos)){
    x2[i,j] <- 1- abs(numberOfEvents[i]-numberOfEvents[j])
  }
}

In [66]:
nodos <- nodos %>%
        mutate(Number.of.Funding.Rounds = ifelse(is.na(nodos$Number.of.Funding.Rounds), mean(nodos$Number.of.Funding.Rounds, na.rm = TRUE), nodos$Number.of.Funding.Rounds))
numberOfRounds <- (nodos$Number.of.Funding.Rounds-min(nodos$Number.of.Funding.Rounds))/(max(nodos$Number.of.Funding.Rounds)-min(nodos$Number.of.Funding.Rounds))
x3 <- matrix(0,nrow(nodos),nrow(nodos))
for(i in 1:nrow(nodos)){
  for(j in 1:nrow(nodos)){
    x3[i,j] <- 1- abs(numberOfRounds[i]-numberOfRounds[j])
  }
}

In [67]:
nodos <- nodos %>%
        mutate(Number.of.Investors = ifelse(is.na(nodos$Number.of.Investors), mean(nodos$Number.of.Investors, na.rm = TRUE), nodos$Number.of.Investors))
numberOfInvestors <- (nodos$Number.of.Investors-min(nodos$Number.of.Investors))/(max(nodos$Number.of.Investors)-min(nodos$Number.of.Investors))
x4 <- matrix(0,nrow(nodos),nrow(nodos))
for(i in 1:nrow(nodos)){
  for(j in 1:nrow(nodos)){
    x4[i,j] <- 1- abs(numberOfInvestors[i]-numberOfInvestors[j])
  }
}

In [79]:
y <- matrix(0,nrow(nodos),nrow(nodos))
g<-0
h <- 0
for (i in nodos$Organization.Name) {
  h <- 0
  elfrom = arcos %>% filter(arcos$Source == i)
  for (j in nodos$Organization.Name) {
    h <- h+1
      if (j %in% elfrom$Target) {
        y[g, h] <- 1

      }
  }
  g <- g+1
}

## Modelo

In [80]:
qap_fit <-netlogit(y,list(x1,x2,x3,x4),rep = 50)

summary(qap_fit)


Network Logit Model

Coefficients:
            Estimate   Exp(b)       Pr(<=b) Pr(>=b) Pr(>=|b|)
(intercept) -7.0619450 0.0008571094 0.00    1.00    0.02     
x1          -0.3330025 0.7167683899 0.32    0.68    0.80     
x2           1.6811521 5.3717411467 0.82    0.18    0.26     
x3          -1.6847534 0.1854901655 0.00    1.00    0.00     
x4           1.1719097 3.2281515305 0.90    0.10    0.28     

Goodness of Fit Statistics:

Null deviance: 31818.23 on 22952 degrees of freedom
Residual deviance: 669.5126 on 22947 degrees of freedom
Chi-Squared test of fit improvement:
	 31148.72 on 5 degrees of freedom, p-value 0 
AIC: 679.5126 	BIC: 719.7184 
Pseudo-R^2 Measures:
	(Dn-Dr)/(Dn-Dr+dfn): 0.5757542 
	(Dn-Dr)/Dn: 0.9789582 
Contingency Table (predicted (rows) x actual (cols)):

        0       1
0   22905      47
1       0       0

	Total Fraction Correct: 0.9979522 
	Fraction Predicted 1s Correct: NaN 
	Fraction Predicted 0s Correct: 0.9979522 
	False Negative Rate: 1 
	False Posi

## Evaluar

In [82]:
exp(coef(qap_fit))

[1] 0.0008571094 0.7167683899 5.3717411467 0.1854901655 3.2281515305

# ERGM

In [83]:
red <- as.network.matrix(y)

## Modelo

In [ ]:
mergm <- ergm(red~edges)

summary(mergm)
exp(coef(mergm))

## Evaluación

In [ ]:
# Correr el GOF
gfergm <- gof(mergm)
par(mfrow = c(3,2))
plot(gfergm, main = "")

# STERGM

In [ ]:
red1 <- 0
red2 <- 0
red3 <- 0

## Modelo

In [ ]:
red = list(red1
           ,red2
           ,red3
           )

fstergm <- stergm(red,
                  formation=~edges+
                    mutual
                    #+gwesp(0.5,fixed=T)
                    #+transitiveties
                    #+density
                    #+ostar(2)
                    +istar(2)
                  ,
                  dissolution=~edges+
                    mutual+
                    nodematch("group", diff = FALSE)+
                    nodematch("cloisterville", diff = FALSE)
                    #+gwesp(0.5,fixed=T)
                    #+transitiveties
                    #+density
                    #+ostar(2)
                    +istar(2)
                  ,
                  estimate = "CMLE",
                  control=control.stergm(SA.plot.progress=TRUE))
summary(fstergm)
exp(coef(fstergm))

## Evaluación